<a href="https://colab.research.google.com/github/NamishBansal15/transformer-modeling-25/blob/main/EfficientDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# -----------------------------------------------
# Fast EfficientDet-style Substation Detector - Fixed Version
# -----------------------------------------------

import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import os
import json
from sklearn.utils import shuffle

# ----------------------------
# CONFIGURATION
# ----------------------------
CONFIG = {
    'model_name': 'efficientnet-b0',
    'num_classes': 3,  # reactor, transformer, circuit_breaker
    'image_size': 224,
    'batch_size': 16,
    'epochs': 50,
    'learning_rate': 0.001,
    'data_path': '/content/',
    'annotation_file': '/content/_annotations.coco.json',
    'max_objects_per_image': 5
}

# ----------------------------
# Data Generator
# ----------------------------
class OptimizedSubstationDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_data, batch_size, image_size, num_classes, max_objects):
        self.image_data = image_data
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.max_objects = max_objects
        self.indexes = np.arange(len(image_data))
        self.on_epoch_end()

    def __len__(self):
        return max(1, len(self.image_data) // self.batch_size)

    def on_epoch_end(self):
        self.indexes = shuffle(self.indexes)

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        if len(batch_indexes) == 0:
            batch_indexes = [0]

        batch_images = []
        batch_classifications = []
        batch_regressions = []

        for i in batch_indexes:
            if i >= len(self.image_data):
                i = 0

            image = self.load_image(self.image_data[i]['filename'])
            labels = self.load_annotations(self.image_data[i])

            batch_images.append(image)

            if labels['classes'] and labels['boxes']:
                dominant_class = max(set(labels['classes']), key=labels['classes'].count)
                boxes = np.array(labels['boxes'])
                avg_box = np.mean(boxes, axis=0)

                batch_classifications.append(dominant_class)
                batch_regressions.append(avg_box)
            else:
                batch_classifications.append(0)
                batch_regressions.append([0.1, 0.1, 0.2, 0.2])

        return (
            np.array(batch_images),
            {
                'classification': np.array(batch_classifications),
                'regression': np.array(batch_regressions)
            }
        )

    @tf.function
    def load_image(self, image_path):
        full_path = os.path.join(CONFIG['data_path'], image_path)
        try:
            image = tf.io.read_file(full_path)
            if full_path.lower().endswith('.png'):
                image = tf.image.decode_png(image, channels=3)
            else:
                image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.resize(image, [self.image_size, self.image_size])
            image = tf.cast(image, tf.float32) / 255.0
            return image
        except Exception as e:
            print(f"Error loading image {full_path}: {e}")
            return tf.zeros((self.image_size, self.image_size, 3), dtype=tf.float32)

    def load_annotations(self, annotation):
        boxes = []
        classes = []

        if 'objects' not in annotation or not annotation['objects']:
            return {'boxes': [], 'classes': []}

        img_width = annotation.get('width', 1)
        img_height = annotation.get('height', 1)

        for obj in annotation['objects']:
            if 'bbox' not in obj:
                continue

            x, y, w, h = obj['bbox']
            if w <= 0 or h <= 0:
                continue

            x1 = max(0, x / img_width)
            y1 = max(0, y / img_height)
            x2 = min(1, (x + w) / img_width)
            y2 = min(1, (y + h) / img_height)

            if x2 <= x1 or y2 <= y1:
                continue

            boxes.append([y1, x1, y2, x2])

            class_map = {'reactor': 0, 'transformer': 1, 'circuit_breaker': 2}
            class_name = obj.get('class', obj.get('category', 'unknown'))
            class_name = class_name.strip().lower().replace(" ", "_")
            if class_name in class_map:
                classes.append(class_map[class_name])
            else:
                print(f"Warning: Unknown class '{class_name}', using default class 0")
                classes.append(0)

        return {'boxes': boxes, 'classes': classes}

# ----------------------------
# Parse COCO
# ----------------------------
def parse_coco_annotations(coco_data):
    print("Parsing COCO annotations...")
    images_dict = {img['id']: img for img in coco_data['images']}
    categories_dict = {cat['id']: cat['name'] for cat in coco_data['categories']}

    image_annotations = {}
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        if image_id not in image_annotations:
            image_annotations[image_id] = []
        image_annotations[image_id].append(ann)

    processed_data = []
    for image_id, image_info in images_dict.items():
        objects = []
        if image_id in image_annotations:
            for ann in image_annotations[image_id]:
                bbox = ann['bbox']
                if len(bbox) == 4 and bbox[2] > 0 and bbox[3] > 0:
                    obj = {
                        'class': categories_dict[ann['category_id']],
                        'bbox': bbox
                    }
                    objects.append(obj)

        processed_data.append({
            'filename': image_info['file_name'],
            'width': image_info['width'],
            'height': image_info['height'],
            'objects': objects
        })

    return processed_data

# ----------------------------
# Model
# ----------------------------
def create_fast_detection_model(num_classes, image_size):
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(image_size, image_size, 3),
        weights='imagenet',
        include_top=False
    )
    base_model.trainable = False

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    class_branch = tf.keras.layers.Dense(128, activation='relu')(x)
    class_branch = tf.keras.layers.Dropout(0.2)(class_branch)
    classification_output = tf.keras.layers.Dense(num_classes, activation='softmax', name='classification')(class_branch)

    reg_branch = tf.keras.layers.Dense(128, activation='relu')(x)
    reg_branch = tf.keras.layers.Dropout(0.2)(reg_branch)
    regression_output = tf.keras.layers.Dense(4, activation='sigmoid', name='regression')(reg_branch)

    model = tf.keras.Model(inputs=base_model.input, outputs=[classification_output, regression_output])
    return model

# ----------------------------
# Train
# ----------------------------
def train_efficientdet():
    print("Starting training...")

    with open(CONFIG['annotation_file'], 'r') as f:
        coco_data = json.load(f)

    processed_data = parse_coco_annotations(coco_data)
    split_idx = int(0.8 * len(processed_data))
    train_data = processed_data[:split_idx]
    val_data = processed_data[split_idx:]

    train_generator = OptimizedSubstationDataGenerator(
        train_data, CONFIG['batch_size'], CONFIG['image_size'], CONFIG['num_classes'], CONFIG['max_objects_per_image'])
    val_generator = OptimizedSubstationDataGenerator(
        val_data, CONFIG['batch_size'], CONFIG['image_size'], CONFIG['num_classes'], CONFIG['max_objects_per_image'])

    model = create_fast_detection_model(CONFIG['num_classes'], CONFIG['image_size'])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(CONFIG['learning_rate']),
        loss={'classification': 'sparse_categorical_crossentropy', 'regression': 'mse'},
        loss_weights={'classification': 1.0, 'regression': 0.5},
        metrics={'classification': ['accuracy'], 'regression': ['mae']}
    )

    model.summary()

    callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss'),
        ModelCheckpoint('efficientdet_substation_best.h5', save_best_only=True, monitor='val_loss'),
        ReduceLROnPlateau(patience=5, factor=0.5, monitor='val_loss', min_lr=1e-7)
    ]

    history = model.fit(
        train_generator,
        epochs=CONFIG['epochs'],
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=1
    )

    model.save('efficientdet_substation_final.h5')

    print("\nFine-tuning...")
    model.layers[0].trainable = True
    model.compile(
        optimizer=tf.keras.optimizers.Adam(CONFIG['learning_rate'] * 0.1),
        loss={'classification': 'sparse_categorical_crossentropy', 'regression': 'mse'},
        loss_weights={'classification': 1.0, 'regression': 0.5},
        metrics={'classification': ['accuracy'], 'regression': ['mae']}
    )

    history_finetune = model.fit(
        train_generator,
        epochs=10,
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=1
    )

    model.save('efficientdet_substation_finetuned.h5')
    return model, history, val_generator

# ----------------------------
# Evaluate
# ----------------------------
def evaluate_model(model, val_generator):
    results = model.evaluate(val_generator, verbose=1)
    print("\n✅ Final Evaluation:")
    print(f"  ➜ Total Loss: {results[0]:.4f}")
    print(f"  ➜ Classification Loss: {results[1]:.4f}")
    print(f"  ➜ Regression Loss: {results[2]:.4f}")
    print(f"  ➜ Classification Accuracy: {results[3]*100:.2f}%")
    print(f"  ➜ Regression MAE: {results[4]:.4f}")

# ----------------------------
# Run
# ----------------------------
if __name__ == "__main__":
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"Using GPU: {gpus}")
        except RuntimeError as e:
            print(f"GPU setup error: {e}")
    else:
        print("No GPU found, using CPU.")

    if not os.path.exists(CONFIG['annotation_file']):
        print(f"Annotation file not found at {CONFIG['annotation_file']}")
        exit(1)
    if not os.path.exists(CONFIG['data_path']):
        print(f"Data path not found at {CONFIG['data_path']}")
        exit(1)

    model, history, val_generator = train_efficientdet()
    evaluate_model(model, val_generator)


No GPU found, using CPU.
Starting training...
Parsing COCO annotations...


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_8         │ (None, 224, 224,  │          0 │ input_layer_4[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_4     │ (None, 224, 224,  │          7 │ rescaling_8[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_9         │ (None, 224, 224,  │          0 │ normalization_4[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_9[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,445,226 (16.96 MB)

 Trainable params: 395,143 (1.51 MB)

...

5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - classification_accuracy: 0.2898 - classification_loss: 1.2004 - loss: 1.2266 - regression_loss: 0.0523 - regression_mae: 0.1850 - val_classification_accuracy: 0.5000 - val_classification_loss: 0.9824 - val_loss: 0.9918 - val_regression_loss: 0.0188 - val_regression_mae: 0.1088 - learning_rate: 0.0010
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - classification_accuracy: 0.4639 - classification_loss: 1.1472 - loss: 1.1637 - regression_loss: 0.0331 - regression_mae: 0.1428 - val_classification_accuracy: 0.4375 - val_classification_loss: 1.0075 - val_loss: 1.0157 - val_regression_loss: 0.0166 - val_regression_mae: 0.0997 - learning_rate: 0.0010
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - classification_accuracy: 0.4306 - classification_loss: 1.0684 - loss: 1.0828 - regression_loss: 0.0286 - regression_mae: 0.1295

5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - classification_accuracy: 0.4339 - classification_loss: 1.0675 - loss: 1.0818 - regression_loss: 0.0286 - regression_mae: 0.1294 - val_classification_accuracy: 0.5625 - val_classification_loss: 0.9045 - val_loss: 0.9105 - val_regression_loss: 0.0120 - val_regression_mae: 0.0879 - learning_rate: 0.0010
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - classification_accuracy: 0.5345 - classification_loss: 1.0663 - loss: 1.0789 - regression_loss: 0.0251 - regression_mae: 0.1232 - val_classification_accuracy: 0.4375 - val_classification_loss: 1.0785 - val_loss: 1.0847 - val_regression_loss: 0.0125 - val_regression_mae: 0.0871 - learning_rate: 0.0010
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - classification_accuracy: 0.5524 - classification_loss: 0.8931 - loss: 0.9063 - regression_loss: 0.0263 - regression_mae: 0.1280 - val_classification_accuracy: 0.3750 - val_classification_loss: 1.2207 - val_loss: 1.2291 - val_regression_loss: 0.0168 - val_regre

5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - classification_accuracy: 0.5550 - classification_loss: 0.9255 - loss: 0.9385 - regression_loss: 0.0260 - regression_mae: 0.1266 - val_classification_accuracy: 0.4375 - val_classification_loss: 0.8924 - val_loss: 0.8982 - val_regression_loss: 0.0116 - val_regression_mae: 0.0840 - learning_rate: 5.0000e-04
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - classification_accuracy: 0.4847 - classification_loss: 0.9710 - loss: 0.9816 - regression_loss: 0.0213 - regression_mae: 0.1093 - val_classification_accuracy: 0.5000 - val_classification_loss: 0.9740 - val_loss: 0.9800 - val_regression_loss: 0.0119 - val_regression_mae: 0.0796 - learning_rate: 5.0000e-04
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - classification_accuracy: 0.5087 - classification_loss: 1.0256 - loss: 1.0376 - regression_loss: 0.0240 - regression_mae: 0.1217 - val_classification_accuracy: 0.5000 - val_classification_loss: 0.9311 - val_loss: 0.9356 - val_regression_loss: 0.0090


Fine-tuning...
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - classification_accuracy: 0.4767 - classification_loss: 0.9636 - loss: 0.9743 - regression_loss: 0.0216 - regression_mae: 0.1154 - val_classification_accuracy: 0.3125 - val_classification_loss: 0.9887 - val_loss: 0.9944 - val_regression_loss: 0.0114 - val_regression_mae: 0.0784 - learning_rate: 1.0000e-04
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - classification_accuracy: 0.4819 - classification_loss: 0.9991 - loss: 1.0104 - regression_loss: 0.0226 - regression_mae: 0.1171 - val_classification_accuracy: 0.5000 - val_classification_loss: 0.9792 - val_loss: 0.9849 - val_regression_loss: 0.0113 - val_regression_mae: 0.0779 - learning_rate: 1.0000e-04
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - classification_accuracy: 0.4750 - classification_loss: 1.0784 - loss: 1.0894 - regression_loss: 0.0221 - regression_mae: 0.1155 - val_classification_accuracy: 0.4375 - val_classification_loss: 0.9886 - val_loss: 0.9940 - val

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - classification_accuracy: 0.5000 - classification_loss: 0.9741 - loss: 0.9795 - regression_loss: 0.0108 - regression_mae: 0.0811

✅ Final Evaluation:
  ➜ Total Loss: 0.9795
  ➜ Classification Loss: 0.9741
  ➜ Regression Loss: 0.0108
  ➜ Classification Accuracy: 50.00%
  ➜ Regression MAE: 0.0811
